In [1]:
!pip install wandb
import wandb
# Replace with your actual API key
api_key = "17dab9d1bbdc37c41831799a4b0b50d3e97400c5"
project_name='RWBCS1'
# Login to Weights & Biases
wandb.login(key=api_key)

wandb: Currently logged in as: rishi1906 (iitm_aero). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import gym
import random
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16, 10)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
torch.manual_seed(0)
import base64
import io  # For visualization
from gym.wrappers.monitoring import video_recorder
from IPython.display import HTML
from IPython import display
import glob
import datetime

In [3]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
env = gym.make('CartPole-v1')
env.seed(0)
state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n
# print('observation space:', env.observation_space)
# print('action space:', env.action_space)


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(


In [5]:

class Policy(nn.Module):
    def __init__(self, state_size=4, action_size=2, hidden_size=32):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = self.fc2(x)
        # we just consider 1 dimensional probability of action
        return F.softmax(x, dim=1)

    def act(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        probs = self.forward(state).cpu()
        model = Categorical(probs)
        action = model.sample()
        return action.item(), model.log_prob(action)


In [6]:

def reinforce(env, policy, optimizer, n_episodes=2000, max_t=1000, gamma=0.99, print_every=100):
    scores_window = deque(maxlen=100)
    scores = []
    average_scores_epsgrdy = []
    episode_list_epsgrdy = []
    for e in range(1, n_episodes):
        saved_log_probs = []
        rewards = []
        state = env.reset()
        # Collect trajectory
        for t in range(max_t):
            # Sample the action from current policy
            action, log_prob = policy.act(state)
            saved_log_probs.append(log_prob)
            state, reward, done, _ = env.step(action)
            rewards.append(reward)
            if done:
                break
        # Calculate total expected reward
        scores_window.append(sum(rewards))
        scores.append(sum(rewards))

        average_score = np.mean(scores_window)
        average_scores_epsgrdy.append(average_score)

        wandb.log({'average_score': average_score})

        episode_list_epsgrdy.append(e)


        # Recalculate the total reward applying discounted factor
        discounts = [gamma ** i for i in range(len(rewards) + 1)]
        R = sum([a * b for a,b in zip(discounts, rewards)])

        # Calculate the loss
        policy_loss = []
        for t, log_prob in enumerate(saved_log_probs):
            # Note that we are using Gradient Ascent, not Descent. So we need to calculate it with negative rewards.
            policy_loss.append(-log_prob * R )
        # After that, we concatenate whole policy loss in 0th dimension
        policy_loss = torch.cat(policy_loss).sum()

        # Backpropagation
        optimizer.zero_grad()
        policy_loss.backward()
        optimizer.step()

        if e % print_every == 0:
            print('Episode {}\tAverage Score: {:.2f}'.format(e, np.mean(scores_window)))
        if np.mean(scores_window) >= 195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(e - 100, np.mean(scores_window)))
            break
    return episode_list_epsgrdy, average_scores_epsgrdy




In [7]:
wandb.init(project=project_name)

In [9]:
sweep_config = {
    'method': 'grid',
    'metric': {
      'name': 'Average Reward',
      'goal': 'maximize'
    },
    'parameters': {
        'state_size': {
            'values': [state_shape]
        },
        'action_size': {
            'values': [action_shape]
        },
        'hidden_size': {
            'values': [32,64,128]
        },
        'max_t': {
            'values': [500, 1000, 1500]
        },
        'lr': {
            'values': [1e-2, 1e-3,1e-4]
        },
        'gamma': {
            'values': [0.99]
        },
        'alpha': {
            'values': [0.01, 0.001, 0.0001]
        },
        'n_episodes': {
            'values': [5000]
        },
        "max_t":{
            'values': [1000]
        },
         "print_every": {
             'values': [100]
         }
    }
}

In [10]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config,  project=project_name)
max_sweep_run = 10 #update it

Create sweep with ID: ud26lmsz
Sweep URL: https://wandb.ai/iitm_aero/RWBCS1/sweeps/ud26lmsz


In [11]:
def main():
    with wandb.init() as run:
        # Get the hyperparameters for this run
        config = wandb.config

        # begin_time = datetime.datetime.now()


        # Create the policy network with the specified hyperparameters
        policy = Policy(state_size=4, action_size=2, hidden_size=config.hidden_size).to(device)
        optimizer = optim.Adam(policy.parameters(), lr=config.lr)

        # Run the REINFORCE algorithm with the specified hyperparameters
        episode_list_epsgrdy, average_scores_epsgrdy = reinforce(policy, optimizer, n_episodes=config.n_episodes, max_t=config.max_t, gamma=config.gamma, print_every=config.print_every)

        # time_taken = datetime.datetime.now() - begin_time

        # print(time_taken)
        # Log the final average score to wandb
        # wandb.log({"Average Score": average_scores_epsgrdy})
        # wandb.log({"Average Score": cumulative_regret})



In [ ]:

wandb.agent(sweep_id, main, count=max_sweep_run)
wandb.finish()